# Дополнительный функционал (настройки)

In [1]:
N = 10 # число запусков для подсчёта среднего времени

## Imports

In [2]:
import time # для подсчёта времени работы
import csv # для сохранения ответов
import pandas as pd # для вывода таблицы
import random # для рандомизированного алгоритма

import warnings # для скрития предупреждения о deprecated
warnings.simplefilter(action='ignore', category=FutureWarning)

## Вспомогательные функции (не участвуют в коде эвристик)

In [3]:
# вспомогательная функция для преобразования ответа к необходимому виду
def transform_answer(solution): # на вход solution вида (число цветов, {словарь соответствия вершина - цвет})
    solution_transformed = [solution[0], [[] for x in range(solution[0])]] # заготовка для трансформированного ответа
    for vertex in solution[1].keys(): # идём по вершинам
        solution_transformed[1][solution[1][vertex] - 1].append(vertex) # добавляем их в массив соответствующего им цвета
    
    for color in range(solution[0]): # пройдём по цветам и отсортируем соответствующие им вершины (просто для красоты финального вывода)
        solution_transformed[1][color].sort()
    return solution_transformed

In [4]:
def check_solution(edges: dict, solution): # на вход словари edges с рёбрами и solution вида (число цветов, {словарь соответствия вершина - цвет})
    vertices = list(edges.keys()) # список вершин, что должны быть окрашены

    for color in range(solution[0]): # проходим по цветам
        for v1 in solution[1][color]: # проходим по списку вершин с указанным цветом
            vertices.remove(v1) # удаляем вершину 
            for v2 in edges[v1]: # проходи по смежным вершинам и проверяем, окрашены ли они в тот же цвет, что и взятая ранее вершина
                if v2 in solution[1][color]: # если да, то выкидываем ошибку
                    raise RuntimeError("Two adjacent vertices have the same color!")
    
    if len(vertices): # проверяем, расскрашены ли все вершины (те, что расскрашены — удалены из списка vertices)
        raise RuntimeError("Not all vertices are colored!")

In [60]:
# вспомогательная функция для сохранения ответов в csv формате
def save_solution(solutions): # solutions - словарь всех полученных ответов
    for dataset in solutions.keys(): # идём по тест-кейсам
        with open(f'solutions/{dataset[:-4]}.csv', 'w', newline='') as file:
            writer = csv.writer(file) # создаём объект для записи
            writer.writerow([solutions[dataset]["colors"]]) # сохраняем число цветов
            writer.writerows(solutions[dataset]["solution"]) # сохраняем соответствие вершин - цветам
    
    # сохранение таблицы в csv формате
    table = pd.DataFrame(data = [], columns=["Instance", "Time, sec", "Colors", "Color classes"])
    for dataset in solutions.keys(): # идём по тест-кейсам
        table = table.append({"Instance": dataset, "Time, sec": solutions[dataset]["time"], "Colors": solutions[dataset]["colors"], "Color classes": solutions[dataset]["solution"]}, ignore_index=True)
    table.to_csv("best_solution.csv", index=False)

In [6]:
# вспомогательная функция для вывода таблицы результатов
def show_results(solutions): # solutions - словарь всех полученных ответов
    table = pd.DataFrame(data = [], columns=["Instance", "Time, sec", "Colors", "Color classes"])
    for dataset in solutions.keys(): # идём по тест-кейсам
        table = table.append({"Instance": dataset, "Time, sec": solutions[dataset]["time"], "Colors": solutions[dataset]["colors"], "Color classes": solutions[dataset]["solution"]}, ignore_index=True)
    display(table.style.hide_index())

# Получение данных

In [7]:
files = ["myciel3.col", "myciel7.col", "school1.col", "school1_nsh.col", "anna.col", "miles1000.col", "miles1500.col", "le450_5a.col", "le450_15b.col", "queen11_11.col"] # файлы, на которых должен быть протестирован код

In [8]:
data = {} 
# data - словарь вида 
# {"название датасета" : 
#     {"vertex_num": число вершин, 
#     "edge_num": число рёбер, 
#     "edges": 
#         {словарь вида вершина - список смежных ей вершин}
#     }
#  ...
# }

In [9]:
for file in files:
    data[file] = {"vertex_num": None, "edge_num": None, "edges": {}}
    with open("datasets/" + file, "r") as f: # открываем файл для чтения
        for row in f: # проходим по строкам
            if row[0] == "c": # если строка начинается с буквы "c" - это комментарий, пропускае строку
                continue
            elif row[0] == "p": # если строка начинается с буквы "p" - это описание проблемы, берём из этой строки число вершин и рёбер (последние два числа)
                data[file]["vertex_num"], data[file]["edge_num"] = int(row.split()[-2]), int(row.split()[-1])
            elif row[0] == "e": # если строка начинается с буквы "p" - это вершины, между которыми есть ребро
                v1, v2 = int(row.split()[-2]), int(row.split()[-1]) # запоминаем вершины

                # добавляем связь вершины v1 с v2
                if v1 not in data[file]["edges"].keys(): # если это первое упоминание вершины v1 - создадим для неё список с указанием v2
                    data[file]["edges"][v1] = [v2]
                elif v2 not in data[file]["edges"][v1]: # иначе - просто добавим v2 в список смежных вершин v1
                    data[file]["edges"][v1].append(v2)

                # аналогично, но относительно вершины v2
                if v2 not in data[file]["edges"].keys():
                    data[file]["edges"][v2] = [v1]
                elif v1 not in data[file]["edges"][v2]: # иначе - просто добавим v2 в список смежных вершин v1
                    data[file]["edges"][v2].append(v1)
        data[file]["edges"] = dict(sorted(data[file]["edges"].items())) # отсортируем вершины

In [10]:
data["myciel3.col"] # пример данных

{'vertex_num': 11,
 'edge_num': 20,
 'edges': {1: [2, 4, 7, 9],
  2: [1, 3, 6, 8],
  3: [2, 5, 7, 10],
  4: [1, 5, 6, 10],
  5: [3, 4, 8, 9],
  6: [2, 4, 11],
  7: [1, 3, 11],
  8: [2, 5, 11],
  9: [1, 5, 11],
  10: [3, 4, 11],
  11: [6, 7, 8, 9, 10]}}

# Реализация эвристик

### Вариант первый (Greedy sequential coloring) - просто раскрашиваем вершины одну за одной в минимально допустимое число цветов.

In [11]:
def sequential_coloring(edges: dict): # на вход - словарь смежных вершине вершин
    # памятка - число цветов не может превышать число вершин
    colors = [1] # цвета для окраски графа
    solution = {} # ответ

    uncolored_vertices = edges.keys() # список неокрашенных вершин

    for v1 in uncolored_vertices: # идём по неокрашенным вершинам
        possible_colors = colors.copy() # создаём список доступных для окраски v1 цветов

        for v2 in edges[v1]: # идём по смежным вершинам
            if solution.get(v2) is not None and solution[v2] in possible_colors: # проверяем, окрашены ли они
                possible_colors.remove(solution[v2]) # если окрашены, то удаляем твкой цвет из допустимых
        if len(possible_colors): # окрашиваем вершину в минимальный доступный цвет, если он имеется
            solution[v1] = possible_colors[0]
        else: # иначе - добавляем новый цвет и окрашиваем в него
            colors.append(colors[-1] + 1)
            solution[v1] = colors[-1]
    
    return len(colors), solution # возвращаем число цветов и соответствие вершин этим цветам

#### Тестирование

In [12]:
solutions_1 = {} 
# словарь для ответов первого варианта жадной эвристики вида
# {"название датасета" : 
#     {"colors": число цветов для окрашивания,
#      "solution": [массивы выршин, соответствующие каждому цвету],
#      "time": время на подсчёт
#     }
# }

for dataset in data.keys(): # идём по тест-кейсам
    time_start = time.time() # замеряем время начала выполнения
    for i in range(N): # делаем 10 запусков для усреднения времени
        sol = sequential_coloring(data[dataset]["edges"]) # считаем число цветов
    time_end = time.time() - time_start # считаем, сколько работал алгоритм
    # print("original solution", sol)
    sol = transform_answer(sol) # преобразовываем ответ к необходимому виду
    check_solution(data[dataset]["edges"], sol) # проверка решения
    # print("transformed solution", sol)
    solutions_1[dataset] = {"colors": sol[0], "solution": sol[1], "time": time_end/N}

#### Вывод результатов

In [13]:
show_results(solutions_1)

Instance,"Time, sec",Colors,Color classes
myciel3.col,0.000000,4,"[[1, 3, 6, 8], [2, 4, 7, 9], [5, 10], [11]]"
myciel7.col,0.000901,8,"[[1, 3, 6, 8, 12, 14, 17, 19, 24, 26, 29, 31, 35, 37, 40, 42, 48, 50, 53, 55, 59, 61, 64, 66, 71, 73, 76, 78, 82, 84, 87, 89, 96, 98, 101, 103, 107, 109, 112, 114, 119, 121, 124, 126, 130, 132, 135, 137, 143, 145, 148, 150, 154, 156, 159, 161, 166, 168, 171, 173, 177, 179, 182, 184], [2, 4, 7, 9, 13, 15, 18, 20, 25, 27, 30, 32, 36, 38, 41, 43, 49, 51, 54, 56, 60, 62, 65, 67, 72, 74, 77, 79, 83, 85, 88, 90, 97, 99, 102, 104, 108, 110, 113, 115, 120, 122, 125, 127, 131, 133, 136, 138, 144, 146, 149, 151, 155, 157, 160, 162, 167, 169, 172, 174, 178, 180, 183, 185], [5, 10, 16, 21, 28, 33, 39, 44, 52, 57, 63, 68, 75, 80, 86, 91, 100, 105, 111, 116, 123, 128, 134, 139, 147, 152, 158, 163, 170, 175, 181, 186], [11, 22, 34, 45, 58, 69, 81, 92, 106, 117, 129, 140, 153, 164, 176, 187], [23, 46, 70, 93, 118, 141, 165, 188], [47, 94, 142, 189], [95, 190], [191]]"
school1.col,0.008400,42,"[[1, 3, 18, 28, 35, 40, 43, 62, 99, 104, 109, 134, 137, 138, 140, 142, 156, 167, 185, 187, 193, 199, 229, 235, 285, 309, 327, 339], [2, 4, 20, 32, 38, 44, 57, 58, 67, 94, 114, 116, 118, 121, 136, 162, 166, 188, 194, 212, 236, 241, 284, 313, 315, 316, 328, 354, 355], [5, 22, 27, 29, 33, 48, 59, 68, 86, 100, 112, 130, 132, 141, 164, 176, 258, 314, 360, 362], [6, 19, 30, 39, 51, 66, 85, 105, 111, 124, 168, 186, 221, 274, 280, 379], [7, 23, 37, 45, 60, 69, 113, 128, 195, 213, 358, 361, 381], [8, 15, 24, 31, 61, 64, 82, 115, 117, 119, 135, 158, 173, 352, 382], [9, 13, 17, 21, 41, 55, 63, 65, 102, 126, 157, 171, 380], [10, 25, 49, 53, 71, 75, 108, 133, 143, 262, 383], [11, 26, 52, 70, 76, 81, 106, 122, 153, 159, 345], [12, 34, 56, 72, 78, 80, 110, 160, 273, 279], [14, 36, 73, 79, 83, 95, 139, 147, 196, 208, 224, 357], [16, 50, 92, 144, 169, 201, 211, 216, 263, 302, 353], [42, 47, 74, 172, 178, 189, 209], [46, 54, 77, 89, 107, 174, 197, 210], [84, 93, 120, 170, 198, 215, 264, 286], [87, 96, 146, 155, 165, 183, 190, 219, 283, 359], [88, 97, 123, 161, 200, 220, 226, 350], [90, 98, 154, 184, 192, 207, 227, 244, 260, 294, 312], [91, 101, 131, 148, 163, 181, 203, 217, 261, 275], [103, 125, 175, 191, 204, 218, 232], [127, 145, 179, 214, 230, 249, 265, 297, 317, 378], [129, 150, 177, 231, 237, 239, 253, 306], [149, 180, 205, 222, 245, 277, 281, 329, 338], [151, 223, 246, 276, 282, 295, 320], [152, 182, 225, 247, 270, 330, 356], [202, 206, 248, 272, 278, 292, 319], [228, 243, 255, 257, 287, 300, 322, 342], [233, 238, 326, 351, 376], [234, 250, 268, 288, 305, 346], [240, 254, 266, 271, 332], [242, 251, 259, 289, 331], [252, 256, 267, 293, 324], [269, 290, 296, 303, 318, 340, 349], [291, 298, 304, 321, 335, 341], [299, 307, 323, 334, 337], [301, 310, 311, 325, 343, 344], [308, 333, 336], [347, 363, 365], [348, 364], [366, 367, 368, 369, 377], [370, 371, 372, 373, 374, 375, 384], [385]]"
school1_nsh.col,0.006306,39,"[[1, 3, 18, 28, 35, 40, 43, 62, 99, 104, 109, 134, 137, 138, 140, 142, 156, 167, 185, 187, 193, 199, 229, 235, 285, 309, 327, 339], [2, 4, 20, 32, 38, 44, 57, 58, 67, 94, 114, 116, 118, 121, 136, 162, 166, 188, 194, 212, 236, 241, 284, 313, 315, 316, 328], [5, 22, 27, 29, 33, 48, 59, 68, 86, 100, 112, 130, 132, 141, 164, 176, 258, 314], [6, 19, 30, 39, 51, 66, 85, 105, 111, 124, 168, 186, 221, 274, 280], [7, 23, 37, 45, 60, 69, 113, 128, 195, 213], [8, 15, 24, 31, 61, 64, 82, 115, 117, 119, 135, 158, 173, 352], [9, 13, 17, 21, 41, 55, 63, 65, 102, 126, 157, 171], [10, 25, 49, 53, 71, 75, 108, 133, 143, 262], [11, 26, 52, 70, 76, 81, 106, 122, 153, 159, 345], [12, 34, 56, 72, 78, 80, 110, 160, 273, 279], [14, 36, 73, 79, 83, 95, 139, 147, 196, 208, 224], [16, 50, 92, 144, 169, 201, 211, 216, 263, 302], [42, 47, 74, 172, 178, 189, 209], [46, 54, 77, 89, 107, 174, 197, 210], [84, 93, 120, 170, 198, 215, 264, 286], [87, 96, 146, 155, 165, 183, 190, 219, 283], [88, 97, 123, 161, 200, 220, 226, 350], [90, 98, 154, 184, 192, 207, 227,

### Вариант второй (Smallest degree last with remove) - сначала отсортировываем все вершины в порядке уменьшения их степеней (с удалением), то есть - изначально рассматриваем вершину с наименьшей степенью (если таких несколько, то берём первую встреченную) и добавляем её в список, как бы полностью убирая её от других вершин (попутно уменьшая их степени). Идём таким образом по всем вершинам, добавляя их в список слева. В итоге получится список вида [вершина с наибольшей степенью, ..., вершина с наименьшей степенью]. Окраску начинаем с вершины наибольшей степени. (ожидается, что его применение даёт результат лучше, чем первого варианта).

In [14]:
def smallest_degree_last_with_remove_v1(edges: dict): # на вход - словарь смежных вершине вершин
    # сортируем smallest degree last with remove
    edges_sorted = {}

    vertices = edges.keys() # список вершин

    vertices_degrees = [len(edges[v]) for v in vertices] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
    vertices_degrees_ids = sorted(range(len(vertices_degrees)), key=vertices_degrees.__getitem__, reverse=True) # создаём список номеров вершин (-1) в порядке уменьшения их степеней
    max_id = len(vertices_degrees_ids) - 1 # максимальное id, до которого можно двигаться вправо в списке

#    print("edges", edges)
#    print("vertices_degrees:    ", vertices_degrees)
#    print("vertices_degrees_ids:", vertices_degrees_ids)

    for id in range(len(vertices)): # просто итерируемся столько раз, сколько у нас вершин
        vertex = vertices_degrees_ids[id] # рассматриваемая вершина (её номер на 1 меньше, чем должен быть из-за нумерации с нуля) с максимальной степенью
        edges_sorted[vertex+1] = edges[vertex+1] # копируем её в новый-отсортированный словарь

        # как бы убираем рассматриваемую вершину из графа
        vertices_degrees[vertex] = -1 # ставим этой вершине степень -1 (для упрощения дальнейшей работы, так как у всех нерассмотренных вершин степень не будет меньше нуля)
        for v in edges[vertex+1]: # идём по смежным вершинам
#            print("before change")
#            print("vertex", vertex, "Рассматриваемая вершина", vertex+1)
#            print("v                 смежная вершина", v)
#            print("vertices_degrees     ", vertices_degrees)
#            print("vertices_degrees_ids:", vertices_degrees_ids)

            vertices_degrees[v-1] -= 1 # понижаем их степени (v-1 так как  нумерация в vertices_degrees идёт с нуля, а не с единицы)

            # а также меняем их позицию в списке vertices_degrees_ids
            v_in_vertices_degrees_ids = vertices_degrees_ids.index(v-1) # находим позицию смежной вершины в списке вершин, отсортированных по убыванию их степеней (чтобы потом относительно неё рассматривать вершины с меньшей степенью)
            
#            print("v_in_vertices_degrees_ids", v_in_vertices_degrees_ids)

            while (v_in_vertices_degrees_ids < max_id) and (-1 < vertices_degrees[v-1] < vertices_degrees[vertices_degrees_ids[v_in_vertices_degrees_ids+1]]): # пока у вершины правее степень больше, сдвигаем рассматриваемую смежную в сторону уменьшения степени

                vertices_degrees_ids[v_in_vertices_degrees_ids], vertices_degrees_ids[v_in_vertices_degrees_ids+1] = vertices_degrees_ids[v_in_vertices_degrees_ids+1], vertices_degrees_ids[v_in_vertices_degrees_ids] # делаем swap вершин в списке, отсортированном по убыванию их степеней
                v_in_vertices_degrees_ids += 1 # переходим к вершине правее (чья степень меньше или равна рассматриваемой смежной)
#            print("after change")
#            print("vertices_degrees     ", vertices_degrees)
#            print("vertices_degrees_ids:", vertices_degrees_ids)
#        print("-------------------------")
#    print("vertices_degrees", vertices_degrees)
#    print("edges_sorted", edges_sorted)
    return sequential_coloring(edges_sorted) # вызываем жадное последовательное окрашивание, но рёбра в нём теперь идут в другом порядке

#### Ещё одна вариация второго алгоритма (работает гораздо быстрее и при этом передаёт немного другую последовательность рёбер)

In [15]:
def smallest_degree_last_with_remove_v2(edges: dict): # на вход - словарь смежных вершине вершин
    # сортируем smallest degree last with remove
    edges_sorted = {}

    vertices = edges.keys() # список вершин

    vertices_degrees = [len(edges[v]) for v in vertices] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
    
    for i in range(len(vertices)): # просто итерируемся столько раз, сколько у нас вершин
        vertex = vertices_degrees.index(max(vertices_degrees)) # берём вершину с самой большой степенью
        vertices_degrees[vertex] = -1 # ставим малую степень найденной вершине (она больше ни разу не будет выбрана)
        edges_sorted[vertex+1] = edges[vertex+1] # копируем рассматриваемую вершину в новый-отсортированный словарь
        for v in edges[vertex+1]: # идём по смежным вершинам
            vertices_degrees[v-1] -= 1 # понижаем их степени (v-1 так как  нумерация в vertices_degrees идёт с нуля, а не с единицы)
#    print("edges_sorted", edges_sorted)
    return sequential_coloring(edges_sorted) # вызываем жадное последовательное окрашивание, но рёбра в нём теперь идут в другом порядке

#### Тестирование

In [16]:
solutions_2_v1 = {}
for dataset in data.keys(): # идём по тест-кейсам
    time_start = time.time() # замеряем время начала выполнения
    for i in range(N): # делаем 10 запусков для усреднения времени
        sol = smallest_degree_last_with_remove_v1(data[dataset]["edges"]) # считаем число цветов
    time_end = time.time() - time_start # считаем, сколько работал алгоритм
    # print("original solution", sol)
    sol = transform_answer(sol) # преобразовываем ответ к необходимому виду
    check_solution(data[dataset]["edges"], sol) # проверка решения
    # print("transformed solution", sol)
    solutions_2_v1[dataset] = {"colors": sol[0], "solution": sol[1], "time": time_end/N}

In [17]:
show_results(solutions_2_v1)

Instance,"Time, sec",Colors,Color classes
myciel3.col,0.000100,4,"[[1, 3, 11], [2, 5, 7, 10], [4, 8, 9], [6]]"
myciel7.col,0.011200,8,"[[1, 3, 11, 23, 47, 95, 191], [2, 5, 13, 16, 22, 49, 52, 58, 60, 63, 69, 94, 97, 100, 106, 108, 111, 117, 142, 144, 147, 153, 155, 158, 160, 163, 164, 167, 170, 176, 178, 181, 183, 186, 187, 189], [4, 7, 8, 9, 27, 30, 31, 32, 46, 99, 102, 103, 104, 118, 122, 125, 126, 127, 133, 136, 137, 138, 141, 146, 149, 150, 151, 165, 169, 172, 173, 174, 180, 184, 185, 188, 190], [6, 10, 24, 28, 29, 33, 48, 53, 57, 59, 64, 68, 70, 71, 75, 76, 78, 80, 82, 86, 87, 89, 91, 93, 96, 101, 105, 119, 123, 124, 128, 143, 148, 152, 154, 159, 166, 171, 175, 177, 182], [12, 14, 15, 26, 34, 35, 37, 38, 40, 42, 43, 45, 50, 61, 62, 73, 81, 84, 85, 90, 92, 98, 107, 109, 110, 121, 129, 130, 132, 135, 140, 145, 156, 157, 168, 179], [18, 20, 25, 36, 41, 51, 54, 56, 65, 67, 72, 74, 77, 79, 83, 88, 113, 115, 120, 131, 162], [17, 19, 21, 44, 55, 66, 112, 114, 116, 139, 161], [39, 134]]"
school1.col,0.149000,35,"[[13, 86, 104, 105, 137, 138, 162, 168, 188, 194, 229, 236, 241, 285, 313, 357, 361, 362, 363, 370, 371, 372, 373, 374, 375], [99, 136, 140, 185, 193, 284, 314, 315, 316, 353, 354, 355, 364, 365, 366, 367, 368, 369, 376, 377, 378, 380], [6, 40, 66, 91, 96, 109, 124, 152, 183, 195, 230, 234, 250, 261, 305, 326, 329, 351, 360, 383, 384, 385], [5, 22, 28, 44, 117, 119, 149, 153, 159, 186, 233, 238, 309, 328, 382], [12, 19, 53, 100, 114, 116, 118, 121, 141, 142, 155, 176, 187, 197, 237, 331, 359, 381], [14, 18, 52, 65, 107, 110, 145, 179, 228, 255, 273, 279, 352, 379], [2, 3, 21, 39, 54, 125, 158, 171, 198, 231, 240, 265, 306], [7, 20, 32, 56, 85, 111, 120, 157, 232, 252, 340], [64, 103, 127, 144, 178, 182, 191, 201, 253, 327], [17, 35, 50, 123, 160, 180, 200, 302, 323, 339, 347, 358], [16, 24, 30, 33, 62, 80, 135, 151, 154, 173, 203, 235, 239, 276, 282], [37, 57, 108, 122, 169, 190, 251, 318, 330], [10, 34, 94, 126, 143, 156, 210, 222, 246, 272, 278, 301], [31, 113, 129, 148, 161, 165, 202, 211, 243, 263, 338], [25, 79, 181, 192, 242, 254, 259, 320, 344], [38, 42, 115, 128, 175, 189, 209, 293], [61, 87, 97, 133, 247, 275, 277, 281, 319, 356], [47, 60, 132, 223, 249, 268, 288, 297, 308, 321, 343, 346], [59, 75, 82, 93, 112, 166, 245, 307, 325, 342, 345, 348], [27, 43, 71, 77, 83, 92, 150, 226, 287], [36, 46, 101, 196, 206, 219, 286, 295, 311, 324], [55, 74, 78, 84, 95, 227, 274, 280, 322], [26, 48, 72, 106, 134, 146, 164, 258, 291, 336, 341], [29, 88, 98, 131, 199, 248, 264, 266, 292], [45, 81, 139, 147, 214, 221, 257, 269, 290], [1, 9, 41, 68, 102, 167, 177, 208, 298, 304, 335], [69, 70, 89, 260, 270, 317, 332], [49, 51, 58, 76, 163, 174, 212, 218, 262, 296, 310, 349], [11, 67, 184, 204, 244, 267, 271, 289, 294, 303], [8, 15, 73, 205, 215, 225, 283, 299, 334, 337], [4, 63, 130, 170, 256, 333], [90, 172, 207, 217, 220, 300, 312], [23, 216, 224], [213], [350]]"
school1_nsh.col,0.115700,34,"[[5, 22, 54, 86, 104, 105, 113, 124, 137, 138, 142, 153, 159, 183, 195, 229, 234, 236, 238, 241, 285, 309, 313, 328], [2, 14, 91, 96, 117, 136, 155, 168, 186, 193, 233, 237, 284, 314, 315, 316, 326, 351], [3, 19, 40, 44, 75, 80, 100, 112, 121, 147, 161, 176, 188, 194, 228, 231, 243, 245, 344, 348], [13, 77, 84, 94, 119, 141, 152, 154, 162, 170, 185, 187, 235, 250, 329], [9, 18, 52, 65, 102, 110, 145, 179, 242, 255, 279, 347], [6, 21, 31, 39, 51, 99, 109, 125, 140, 150, 156, 171, 240, 306], [7, 20, 53, 108, 123, 139, 191, 232, 252, 261, 340], [4, 30, 33, 116, 127, 151, 165, 180, 239, 253, 263, 343], [35, 57, 73, 107, 111, 120, 146, 167, 178, 256, 323, 327, 331, 339], [37, 78, 83, 114, 118, 133, 144, 169, 201, 211, 217, 251, 318], [16, 49, 50, 59, 122, 246, 287, 321, 342, 345], [11, 34, 56, 87, 115, 126, 143, 200, 202, 210, 254, 272, 278, 332], [12, 38, 45, 98, 128, 157, 192, 197, 209], [103, 129, 158, 174, 182, 189, 196, 198, 247, 311, 352], [17, 24, 43, 61, 66, 132, 134, 148, 258, 301, 338], [42, 81, 149, 181, 190, 205, 208, 257, 290, 322, 3

In [18]:
solutions_2_v2 = {}
for dataset in data.keys(): # идём по тест-кейсам
    time_start = time.time() # замеряем время начала выполнения
    for i in range(N): # делаем 10 запусков для усреднения времени
        sol = smallest_degree_last_with_remove_v2(data[dataset]["edges"]) # считаем число цветов
    time_end = time.time() - time_start # считаем, сколько работал алгоритм
    # print("original solution", sol)
    sol = transform_answer(sol) # преобразовываем ответ к необходимому виду
    check_solution(data[dataset]["edges"], sol) # проверка решения
    # print("transformed solution", sol)
    solutions_2_v2[dataset] = {"colors": sol[0], "solution": sol[1], "time": time_end/N}

#### Вывод результатов

In [19]:
show_results(solutions_2_v2)

Instance,"Time, sec",Colors,Color classes
myciel3.col,0.000000,4,"[[1, 3, 11], [2, 4, 7, 9], [5, 6, 10], [8]]"
myciel7.col,0.002499,8,"[[1, 3, 11, 23, 47, 95, 191], [2, 4, 13, 15, 22, 49, 51, 58, 60, 62, 69, 94, 97, 99, 106, 108, 110, 117, 142, 144, 146, 153, 155, 157, 160, 162, 164, 167, 169, 176, 178, 180, 183, 185, 187, 189], [5, 6, 7, 10, 28, 29, 30, 33, 46, 100, 101, 102, 105, 118, 123, 124, 125, 128, 134, 135, 136, 139, 141, 147, 148, 149, 152, 165, 170, 171, 172, 175, 181, 182, 186, 188, 190], [8, 9, 26, 27, 31, 32, 50, 55, 56, 61, 66, 67, 70, 73, 74, 76, 78, 79, 84, 85, 87, 89, 90, 93, 98, 103, 104, 121, 122, 126, 127, 145, 150, 151, 156, 161, 168, 173, 174, 179, 184], [12, 14, 16, 24, 34, 35, 37, 39, 40, 42, 44, 45, 48, 59, 63, 71, 81, 82, 86, 91, 92, 96, 107, 109, 111, 119, 129, 130, 132, 137, 140, 143, 154, 158, 166, 177], [18, 21, 25, 36, 41, 52, 54, 57, 65, 68, 72, 75, 77, 80, 83, 88, 113, 116, 120, 131, 163], [17, 19, 20, 43, 53, 64, 112, 114, 115, 138, 159], [38, 133]]"
school1.col,0.016400,37,"[[2, 13, 22, 40, 99, 104, 137, 138, 162, 168, 188, 194, 229, 236, 241, 285, 309, 361, 363, 370, 371, 372, 373, 374, 375], [105, 136, 140, 185, 193, 284, 313, 315, 316, 353, 354, 355, 357, 364, 365, 366, 367, 368, 369, 376, 377, 378, 380], [6, 66, 86, 91, 96, 109, 124, 152, 183, 195, 230, 234, 250, 261, 305, 314, 326, 329, 351, 360, 384, 385], [11, 44, 112, 130, 149, 153, 159, 186, 233, 238, 328, 362, 379], [12, 19, 53, 57, 100, 114, 116, 118, 121, 142, 155, 176, 187, 197, 237, 331, 359, 381], [14, 18, 56, 65, 110, 131, 145, 179, 228, 255, 273, 279, 382], [3, 21, 39, 54, 113, 125, 158, 171, 198, 231, 240, 306, 383], [7, 20, 32, 55, 85, 111, 120, 139, 157, 232, 252, 340], [23, 37, 64, 127, 144, 178, 182, 191, 201, 235, 253, 265], [17, 35, 50, 117, 119, 123, 160, 180, 200, 302, 323, 327, 339, 347], [16, 30, 62, 106, 115, 128, 141, 151, 154, 170, 276, 282, 320], [38, 78, 98, 108, 122, 161, 169, 251, 264, 345], [34, 126, 143, 175, 210, 222, 246, 272, 278, 301, 318], [31, 103, 129, 156, 174, 192, 202, 209, 308], [5, 24, 45, 61, 97, 213, 239, 254, 275, 277, 281], [1, 10, 42, 132, 189, 205, 211, 242, 247, 257, 330, 356], [60, 75, 133, 148, 166, 243, 288, 307, 321, 343, 346, 358], [8, 87, 93, 249, 263, 297, 319, 342, 344], [4, 25, 43, 71, 77, 83, 259, 280], [146, 164, 190, 196, 203, 295, 310, 311, 325], [36, 72, 79, 84, 92, 225, 274], [47, 74, 88, 101, 147, 199, 266, 286, 324], [26, 48, 58, 95, 226, 248, 258, 291, 293, 341], [41, 46, 49, 59, 94, 245, 287, 317, 348], [9, 29, 52, 68, 81, 102, 173, 208, 269, 290, 292], [89, 227, 260, 268, 299, 322, 334, 337], [70, 107, 150, 172, 184, 204, 223, 270, 271, 332], [51, 69, 134, 167, 177, 262, 298, 304, 335], [33, 63, 76, 80, 256, 283, 333], [67, 135, 163, 181, 212, 218, 267, 289, 296, 303, 338, 349], [27, 73, 90, 207, 215, 312], [28, 165, 224, 244, 294], [15, 82, 336, 352], [206, 214, 221], [216, 219], [217, 220, 300], [350]]"
school1_nsh.col,0.012801,34,"[[2, 5, 22, 40, 54, 99, 104, 112, 124, 137, 138, 142, 153, 159, 183, 195, 229, 234, 236, 238, 241, 285, 309, 328], [14, 28, 86, 91, 96, 116, 136, 155, 168, 186, 193, 233, 237, 284, 313, 315, 316, 326], [3, 19, 44, 75, 80, 100, 113, 121, 147, 161, 176, 188, 194, 228, 231, 243, 245, 314, 344, 348], [8, 18, 52, 65, 102, 110, 145, 179, 185, 235, 242, 255, 279, 347], [13, 57, 77, 84, 94, 118, 152, 154, 162, 170, 187, 250, 329], [6, 21, 31, 39, 51, 105, 109, 125, 140, 150, 156, 171, 240, 306], [7, 20, 56, 85, 111, 120, 157, 167, 232, 252, 261, 340], [23, 37, 64, 127, 144, 178, 182, 191, 201, 253, 265], [35, 50, 106, 123, 141, 177, 189, 200, 219, 302, 323, 327, 331, 339], [15, 30, 33, 108, 128, 151, 165, 180, 214, 318], [16, 38, 53, 98, 117, 122, 169, 190, 251, 264, 337], [10, 34, 55, 126, 143, 210, 225, 246, 272, 278, 301], [97, 119, 129, 192, 197, 209, 239, 254, 263, 308], [87, 103, 114, 132, 134, 164, 196, 198, 247, 286, 330], [17, 24, 43, 83, 133, 135, 148, 175, 202, 221], [4, 107, 115, 149, 249, 275, 277, 281, 311], [12, 36, 61, 79, 216, 2

        Первый вариант эвристики Smallest degree last with remove требует для выполнения гораздо больше времени, однако его ответы оказались чуть лучше второго варианта.

### Вариант третий (Largest degree first with randomization) — алгоритм схож со вторым, но вместо долгой сортировки вершин с удалением, здесь они будут просто один раз сортироваться в порядке уменьшения степеней вершин и случайно перемешиваться несколько раз (с запуском после каждого перемешивания).

In [37]:
def randomized_largest_degree_first(edges:dict, percent=10, runs=10): # на вход edges - словарь смежных вершине вершин, percent - не больше какого процента вершин будут случайно перемешаны, runs - сколько попыток было без улучшения решения
    vertices = edges.keys() # список вершин

    vertices_degrees = [len(edges[v]) for v in vertices] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
    order = sorted(range(len(vertices_degrees)), key=vertices_degrees.__getitem__, reverse=True) # создаём список номеров вершин (-1) в порядке уменьшения их степеней

    order_best = order.copy() # сохраняем лучший порядок
    sol_best = [len(vertices), None] # начальное решение для
    attempts = 0 # инициализируем число попыток

    while attempts < runs: # запускаем алгоритм, пока число попыток без изменения результата не превысит счётчик runs
        sol = sequential_coloring({v+1:edges[v+1] for v in order}) # передаём сгенерированный словарь рёбер с нужным порядком рассматриваемых вершин
        if sol[0] < sol_best[0]: # если число цветов оказалось меньше, чем было до этого в лучшем ответе
            sol_best = sol # обновляем лучшее решение
            order_best = order.copy() # запоминаем порядок переданных вершин
            attempts = 0 # зануляем счётчик попыток без улучшения

        else: # если новое решение оказалось не лучше
            order = order_best.copy() # возвращаемся к предыдущей лучшей последовательности вершин
            attempts += 1

        for i in range(max(1, int(len(order)%(percent/2))) + attempts): # делаем случайные перестановки в последовательности передаваемых вершин (рёбер), как минимум одна перестановка (с увеличением числа попыток делается больше перестановок)
            i1, i2 = random.sample(order, 2)
            order[i1], order[i2] = order[i2], order[i1]

    return sol_best # возвращаем лучший найденный ответ

#### Тестирование

In [40]:
solutions_3 = {} 

for dataset in data.keys(): # идём по тест-кейсам
    time_start = time.time() # замеряем время начала выполнения
    for i in range(N): # делаем 10 запусков для усреднения времени
        sol = randomized_largest_degree_first(data[dataset]["edges"], percent=10, runs=25) # считаем число цветов
    time_end = time.time() - time_start # считаем, сколько работал алгоритм
    sol = transform_answer(sol) # преобразовываем ответ к необходимому виду
    check_solution(data[dataset]["edges"], sol) # проверка решения
    solutions_3[dataset] = {"colors": sol[0], "solution": sol[1], "time": time_end/N}

#### Вывод результатов

In [41]:
show_results(solutions_3)

Instance,"Time, sec",Colors,Color classes
myciel3.col,0.002200,4,"[[1, 3, 11], [2, 4, 7, 9], [5, 6, 10], [8]]"
myciel7.col,0.029500,8,"[[1, 3, 11, 23, 47, 95, 191], [2, 4, 7, 9, 25, 27, 30, 32, 46, 97, 99, 102, 104, 118, 120, 122, 125, 127, 131, 133, 136, 138, 141, 144, 146, 149, 151, 165, 167, 169, 172, 174, 178, 180, 183, 185, 188, 190], [5, 6, 10, 48, 52, 53, 57, 59, 63, 64, 68, 70, 94, 96, 100, 101, 105, 142, 143, 147, 148, 152, 154, 158, 159, 160, 163, 166, 170, 171, 175, 177, 181, 182, 186, 189], [8, 12, 14, 15, 17, 19, 20, 26, 31, 35, 37, 38, 40, 42, 43, 50, 55, 61, 62, 66, 67, 73, 78, 82, 84, 85, 87, 89, 90, 98, 103, 107, 109, 110, 112, 114, 115, 121, 126, 130, 132, 135, 137, 145, 150, 156, 157, 161, 162, 168, 173, 179, 184], [13, 16, 22, 28, 34, 36, 39, 45, 49, 58, 60, 69, 72, 75, 81, 83, 86, 92, 106, 108, 111, 117, 123, 129, 134, 140, 153, 155, 164, 176, 187], [21, 24, 29, 33, 41, 44, 71, 76, 77, 79, 80, 88, 91, 116, 119, 124, 128, 139], [18, 51, 54, 56, 65, 74, 113], [93]]"
school1.col,0.271600,30,"[[2, 13, 104, 105, 137, 138, 162, 188, 194, 229, 236, 241, 285, 313, 361, 362, 370, 371, 372, 373, 374, 375, 384], [40, 99, 136, 140, 185, 193, 284, 315, 316, 353, 354, 355, 357, 364, 365, 366, 367, 368, 369, 376, 377, 378, 380], [6, 22, 66, 74, 86, 91, 96, 111, 124, 152, 168, 183, 186, 221, 234, 250, 261, 274, 280, 305, 314, 326, 329, 351, 360, 385], [5, 28, 44, 57, 116, 118, 149, 153, 159, 187, 233, 238, 309, 328, 382], [9, 17, 21, 31, 39, 54, 63, 100, 129, 148, 155, 156, 171, 192, 195, 235, 237, 239, 243, 253, 265, 283, 301, 306, 363, 383], [3, 19, 35, 75, 112, 121, 161, 176, 204, 228, 231, 245, 344, 348, 359, 381], [8, 18, 43, 65, 87, 94, 110, 145, 170, 179, 273, 279, 330, 358, 379], [55, 102, 108, 125, 139, 142, 165, 177, 191, 232, 240, 308], [23, 34, 85, 119, 120, 167, 198, 199, 200, 213, 252, 266, 286, 340], [50, 106, 114, 123, 141, 189, 196, 220, 300, 302, 323, 327, 331, 339], [15, 30, 117, 127, 151, 180, 214, 304, 318, 343], [4, 24, 37, 78, 83, 115, 132, 144, 154, 169, 201, 251, 325], [27, 59, 77, 103, 122, 164, 219, 226, 246, 287, 295, 307, 342, 345], [10, 25, 38, 45, 128, 143, 202, 210, 254, 259, 272, 278, 335, 350], [12, 67, 70, 84, 92, 126, 150, 157, 190, 225, 247, 267, 271, 289], [73, 97, 133, 184, 205, 242, 255, 257, 291, 299, 334, 337], [46, 72, 79, 89, 173, 197, 206, 209, 263, 270, 311, 319], [16, 32, 109, 178, 216, 223, 293, 321, 346], [14, 29, 47, 71, 88, 93, 130, 134, 172, 203, 264, 276, 282, 292], [42, 61, 76, 81, 163, 181, 207, 212, 218, 275, 277, 296, 320, 349], [36, 90, 135, 158, 248, 258, 281, 324], [7, 41, 49, 60, 101, 268, 288, 298, 310, 332], [1, 26, 52, 58, 82, 95, 215, 227, 341], [33, 48, 80, 113, 131, 147, 160, 208, 256, 290, 297], [62, 64, 98, 244, 260, 294, 312, 338, 347], [11, 51, 68, 262, 269, 303, 317, 333, 336, 352], [53, 69, 107, 146, 166, 217, 224, 322], [20, 56, 175, 211], [174, 182, 222, 356], [230, 249]]"
school1_nsh.col,0.253399,31,"[[1, 14, 18, 35, 44, 104, 105, 114, 130, 137, 138, 162, 170, 186, 193, 200, 202, 203, 228, 236, 241, 285, 309, 313, 330], [2, 5, 22, 40, 99, 112, 124, 136, 149, 153, 159, 195, 229, 233, 238, 284, 315, 316, 328], [12, 19, 51, 57, 86, 100, 115, 117, 119, 121, 142, 155, 156, 168, 185, 187, 197, 237, 314, 331], [8, 21, 31, 39, 65, 109, 125, 140, 150, 157, 171, 188, 194, 240, 306], [13, 77, 84, 94, 116, 118, 141, 152, 154, 235, 250, 329], [20, 38, 53, 79, 131, 133, 135, 145, 176, 180, 216, 225, 252, 264, 292, 338], [23, 37, 64, 127, 144, 178, 182, 191, 201, 232, 253, 265], [6, 24, 83, 85, 120, 179, 224, 325, 336, 340], [17, 25, 61, 88, 123, 161, 174, 251, 337], [16, 30, 92, 128, 151, 318, 341], [45, 49, 50, 59, 103, 122, 219, 226, 246, 287, 317, 342, 345], [71, 75, 87, 134, 143, 196, 210, 222, 272, 278, 326], [27, 43, 68, 126, 148, 164, 165, 189, 208, 227, 247, 258, 269, 290, 352], [3, 47, 97, 198, 239, 249, 263, 286, 308, 319], [7, 70, 108, 181, 192, 199, 242, 254, 259, 320], [32, 95, 129, 160, 243, 255, 271, 289, 334], [33, 46, 60, 89, 175, 206, 268, 288

        Использование рандомизации принесло свои плоды — большинство ответов заметно улучшились. Однако в кейсе "queen11_11.col" произошло ухудшение — число цветов увеличилось на 1.

### Вариант четвёртый (финальный) - совместить Smallest degree last with remove (второй вариант) с рандомизацией.

In [51]:
def randomized_smallest_degree_last_with_remove(edges: dict, percent=10, runs=10): # на вход edges - словарь смежных вершине вершин, percent - не больше какого процента вершин будут случайно перемешаны, runs - сколько попыток было без улучшения решения
    # сортируем smallest degree last with remove
    order = [] # создаём список номеров вершин (-1) в порядке уменьшения их степеней

    vertices = edges.keys() # список вершин

    vertices_degrees = [len(edges[v]) for v in vertices] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
    
    for i in range(len(vertices)): # просто итерируемся столько раз, сколько у нас вершин
        vertex = vertices_degrees.index(max(vertices_degrees)) # берём вершину с самой большой степенью
        vertices_degrees[vertex] = -1 # ставим малую степень найденной вершине (она больше ни разу не будет выбрана)
        order.append(vertex) # копируем рассматриваемую вершину в новый - отсортированный список
        for v in edges[vertex+1]: # идём по смежным вершинам
            vertices_degrees[v-1] -= 1 # понижаем их степени (v-1 так как  нумерация в vertices_degrees идёт с нуля, а не с единицы)


    order_best = order.copy() # сохраняем лучший порядок
    sol_best = [len(vertices), None] # начальное решение для
    attempts = 0 # инициализируем число попыток

    while attempts < runs: # запускаем алгоритм, пока число попыток без изменения результата не превысит счётчик runs
        sol = sequential_coloring({v+1:edges[v+1] for v in order}) # передаём сгенерированный словарь рёбер с нужным порядком рассматриваемых вершин
        if sol[0] < sol_best[0]: # если число цветов оказалось меньше, чем было до этого в лучшем ответе
            sol_best = sol # обновляем лучшее решение
            order_best = order.copy() # запоминаем порядок переданных вершин
            attempts = 0 # зануляем счётчик попыток без улучшения

        else: # если новое решение оказалось не лучше
            order = order_best.copy() # возвращаемся к предыдущей лучшей последовательности вершин
            attempts += 1

        # for i in range(max(1, int(len(order)%(percent/2))) + attempts): # делаем случайные перестановки в последовательности передаваемых вершин (рёбер), как минимум одна перестановка (с увеличением числа попыток делается больше перестановок)
        for i in range(max(1, int(len(order)%(percent/2)))): # делаем случайные перестановки в последовательности передаваемых вершин (рёбер), как минимум одна перестановка
            i1, i2 = random.sample(order, 2)
            order[i1], order[i2] = order[i2], order[i1]

    return sol_best # возвращаем лучший найденный ответ

#### Тестирование

In [56]:
solutions_4 = {} 

for dataset in data.keys(): # идём по тест-кейсам
    time_start = time.time() # замеряем время начала выполнения
    for i in range(N): # делаем 10 запусков для усреднения времени
        sol = randomized_smallest_degree_last_with_remove(data[dataset]["edges"], percent=10, runs=50) # считаем число цветов
    time_end = time.time() - time_start # считаем, сколько работал алгоритм
    sol = transform_answer(sol) # преобразовываем ответ к необходимому виду
    check_solution(data[dataset]["edges"], sol) # проверка решения
    solutions_4[dataset] = {"colors": sol[0], "solution": sol[1], "time": time_end/N}

#### Вывод результатов

In [57]:
show_results(solutions_4)

Instance,"Time, sec",Colors,Color classes
myciel3.col,0.001100,4,"[[1, 3, 11], [2, 4, 7, 9], [5, 6, 10], [8]]"
myciel7.col,0.044202,8,"[[1, 3, 11, 23, 47, 95, 191], [2, 4, 13, 15, 22, 49, 51, 58, 60, 62, 69, 94, 97, 99, 106, 108, 110, 117, 142, 144, 146, 153, 155, 157, 160, 162, 164, 167, 169, 176, 178, 180, 183, 185, 187, 189], [5, 6, 7, 10, 28, 29, 30, 33, 46, 100, 101, 102, 105, 118, 123, 124, 125, 128, 134, 135, 136, 139, 141, 147, 148, 149, 152, 165, 170, 171, 172, 175, 181, 182, 186, 188, 190], [8, 9, 26, 27, 31, 32, 50, 55, 56, 61, 66, 67, 70, 73, 74, 76, 78, 79, 84, 85, 87, 89, 90, 93, 98, 103, 104, 121, 122, 126, 127, 145, 150, 151, 156, 161, 168, 173, 174, 179, 184], [12, 14, 16, 24, 34, 35, 37, 39, 40, 42, 44, 45, 48, 59, 63, 71, 81, 82, 86, 91, 92, 96, 107, 109, 111, 119, 129, 130, 132, 137, 140, 143, 154, 158, 166, 177], [18, 21, 25, 36, 41, 52, 54, 57, 65, 68, 72, 75, 77, 80, 83, 88, 113, 116, 120, 131, 163], [17, 19, 20, 43, 53, 64, 112, 114, 115, 138, 159], [38, 133]]"
school1.col,0.732221,32,"[[2, 13, 22, 40, 99, 104, 137, 138, 162, 168, 188, 194, 229, 236, 241, 285, 309, 361, 363, 370, 371, 372, 373, 374, 375], [105, 136, 140, 185, 193, 284, 313, 315, 316, 353, 354, 355, 357, 364, 365, 366, 367, 368, 369, 376, 377, 378, 380], [6, 45, 66, 74, 86, 96, 124, 152, 195, 221, 250, 274, 280, 314, 326, 329, 351, 360, 384, 385], [11, 54, 112, 142, 153, 159, 183, 186, 230, 234, 238, 328, 362, 379], [1, 14, 28, 57, 91, 103, 155, 156, 187, 233, 237], [3, 19, 44, 75, 80, 100, 113, 121, 147, 161, 176, 228, 231, 243, 245, 344, 348, 381], [8, 18, 29, 52, 65, 102, 110, 145, 179, 235, 242, 255, 279, 382], [9, 21, 31, 39, 51, 109, 125, 141, 150, 158, 171, 240, 306, 383], [7, 20, 32, 56, 85, 111, 120, 157, 232, 252, 261, 340], [23, 37, 64, 127, 144, 178, 182, 191, 201, 253, 265], [35, 50, 106, 123, 160, 177, 200, 244, 302, 323, 327, 331, 339, 347], [30, 33, 116, 118, 128, 151, 170, 180, 214, 318, 343], [4, 38, 78, 108, 122, 169, 190, 251, 325], [10, 34, 94, 119, 126, 143, 210, 222, 246, 272, 278, 301], [97, 117, 129, 165, 192, 197, 209, 239, 254, 263, 308], [16, 36, 87, 114, 132, 198, 266, 286, 321, 356], [70, 115, 134, 139, 173, 202, 211, 247, 303, 319, 330], [25, 43, 83, 164, 249, 259, 275, 281, 297], [17, 24, 61, 133, 135, 148, 166, 212, 219, 258, 307, 324], [42, 49, 60, 77, 149, 174, 213, 268, 288, 295, 346], [12, 62, 79, 130, 154, 203, 216, 256, 276, 282, 293, 305, 320], [46, 71, 89, 92, 226, 283, 292, 311], [41, 59, 84, 101, 131, 248, 287, 317, 342, 345], [47, 69, 73, 88, 93, 172, 223, 277, 334, 337, 358, 359], [55, 58, 81, 95, 205, 227, 257, 291, 298, 322, 332, 335, 350], [76, 98, 163, 181, 207, 218, 260, 296, 312, 341, 349], [53, 107, 146, 175, 196, 199, 310, 333, 352], [5, 26, 48, 68, 167, 189, 208, 264, 269, 290, 294, 304], [82, 270, 271, 299, 338], [27, 72, 90, 262, 273, 336], [15, 63, 206, 215, 220, 225], [67, 184, 204, 217, 224, 267, 289, 300]]"
school1_nsh.col,0.562799,30,"[[2, 5, 22, 40, 54, 99, 104, 112, 124, 137, 138, 142, 153, 159, 183, 195, 229, 234, 236, 238, 241, 285, 309, 328], [14, 28, 86, 91, 96, 116, 136, 155, 168, 186, 193, 233, 237, 284, 313, 315, 316, 326], [3, 19, 44, 75, 80, 100, 113, 121, 161, 174, 188, 194, 228, 231, 243, 245, 314, 344, 348], [8, 18, 52, 65, 102, 110, 145, 176, 179, 185, 235, 242, 255, 279, 347], [13, 57, 77, 84, 94, 118, 152, 154, 162, 170, 187, 250, 329], [6, 21, 31, 39, 51, 105, 109, 125, 140, 150, 156, 171, 240, 306], [7, 20, 56, 85, 111, 120, 157, 167, 232, 252, 261, 340], [23, 37, 64, 127, 144, 178, 182, 191, 201, 253, 265], [35, 50, 106, 123, 141, 177, 189, 200, 219, 302, 323, 327, 331, 339], [15, 30, 33, 108, 128, 151, 165, 180, 214, 318], [16, 38, 53, 59, 98, 114, 122, 169, 251, 264, 287, 342, 345], [10, 34, 55, 126, 143, 210, 225, 246, 272, 278, 301], [17, 27, 117, 119, 129, 148, 164, 192, 197, 199, 239, 263, 307], [87, 103, 115, 132, 134, 172, 190, 196, 198, 209, 247, 330], [25, 43, 74, 83, 175, 202, 221, 259, 351], [4, 61, 133, 149, 163, 213, 227, 275, 277, 281, 295, 31

        Четвёртый вариант алгоритма показал схожие результаты с третьим. Где-то ответы хуже, где-то лучше (это связано с элементом случайности в этих алгоритмах). Но они оба в целом дают довольно хорошие ответы.

# Вывод

        Использование рандомизации — наше всё!

        Лучшие ответы получились при использовании Smallest degree last with remove и Largest degree first с элементом случайности.

In [62]:
show_results(solutions_4)

Instance,"Time, sec",Colors,Color classes
myciel3.col,0.001100,4,"[[1, 3, 11], [2, 4, 7, 9], [5, 6, 10], [8]]"
myciel7.col,0.044202,8,"[[1, 3, 11, 23, 47, 95, 191], [2, 4, 13, 15, 22, 49, 51, 58, 60, 62, 69, 94, 97, 99, 106, 108, 110, 117, 142, 144, 146, 153, 155, 157, 160, 162, 164, 167, 169, 176, 178, 180, 183, 185, 187, 189], [5, 6, 7, 10, 28, 29, 30, 33, 46, 100, 101, 102, 105, 118, 123, 124, 125, 128, 134, 135, 136, 139, 141, 147, 148, 149, 152, 165, 170, 171, 172, 175, 181, 182, 186, 188, 190], [8, 9, 26, 27, 31, 32, 50, 55, 56, 61, 66, 67, 70, 73, 74, 76, 78, 79, 84, 85, 87, 89, 90, 93, 98, 103, 104, 121, 122, 126, 127, 145, 150, 151, 156, 161, 168, 173, 174, 179, 184], [12, 14, 16, 24, 34, 35, 37, 39, 40, 42, 44, 45, 48, 59, 63, 71, 81, 82, 86, 91, 92, 96, 107, 109, 111, 119, 129, 130, 132, 137, 140, 143, 154, 158, 166, 177], [18, 21, 25, 36, 41, 52, 54, 57, 65, 68, 72, 75, 77, 80, 83, 88, 113, 116, 120, 131, 163], [17, 19, 20, 43, 53, 64, 112, 114, 115, 138, 159], [38, 133]]"
school1.col,0.732221,32,"[[2, 13, 22, 40, 99, 104, 137, 138, 162, 168, 188, 194, 229, 236, 241, 285, 309, 361, 363, 370, 371, 372, 373, 374, 375], [105, 136, 140, 185, 193, 284, 313, 315, 316, 353, 354, 355, 357, 364, 365, 366, 367, 368, 369, 376, 377, 378, 380], [6, 45, 66, 74, 86, 96, 124, 152, 195, 221, 250, 274, 280, 314, 326, 329, 351, 360, 384, 385], [11, 54, 112, 142, 153, 159, 183, 186, 230, 234, 238, 328, 362, 379], [1, 14, 28, 57, 91, 103, 155, 156, 187, 233, 237], [3, 19, 44, 75, 80, 100, 113, 121, 147, 161, 176, 228, 231, 243, 245, 344, 348, 381], [8, 18, 29, 52, 65, 102, 110, 145, 179, 235, 242, 255, 279, 382], [9, 21, 31, 39, 51, 109, 125, 141, 150, 158, 171, 240, 306, 383], [7, 20, 32, 56, 85, 111, 120, 157, 232, 252, 261, 340], [23, 37, 64, 127, 144, 178, 182, 191, 201, 253, 265], [35, 50, 106, 123, 160, 177, 200, 244, 302, 323, 327, 331, 339, 347], [30, 33, 116, 118, 128, 151, 170, 180, 214, 318, 343], [4, 38, 78, 108, 122, 169, 190, 251, 325], [10, 34, 94, 119, 126, 143, 210, 222, 246, 272, 278, 301], [97, 117, 129, 165, 192, 197, 209, 239, 254, 263, 308], [16, 36, 87, 114, 132, 198, 266, 286, 321, 356], [70, 115, 134, 139, 173, 202, 211, 247, 303, 319, 330], [25, 43, 83, 164, 249, 259, 275, 281, 297], [17, 24, 61, 133, 135, 148, 166, 212, 219, 258, 307, 324], [42, 49, 60, 77, 149, 174, 213, 268, 288, 295, 346], [12, 62, 79, 130, 154, 203, 216, 256, 276, 282, 293, 305, 320], [46, 71, 89, 92, 226, 283, 292, 311], [41, 59, 84, 101, 131, 248, 287, 317, 342, 345], [47, 69, 73, 88, 93, 172, 223, 277, 334, 337, 358, 359], [55, 58, 81, 95, 205, 227, 257, 291, 298, 322, 332, 335, 350], [76, 98, 163, 181, 207, 218, 260, 296, 312, 341, 349], [53, 107, 146, 175, 196, 199, 310, 333, 352], [5, 26, 48, 68, 167, 189, 208, 264, 269, 290, 294, 304], [82, 270, 271, 299, 338], [27, 72, 90, 262, 273, 336], [15, 63, 206, 215, 220, 225], [67, 184, 204, 217, 224, 267, 289, 300]]"
school1_nsh.col,0.562799,30,"[[2, 5, 22, 40, 54, 99, 104, 112, 124, 137, 138, 142, 153, 159, 183, 195, 229, 234, 236, 238, 241, 285, 309, 328], [14, 28, 86, 91, 96, 116, 136, 155, 168, 186, 193, 233, 237, 284, 313, 315, 316, 326], [3, 19, 44, 75, 80, 100, 113, 121, 161, 174, 188, 194, 228, 231, 243, 245, 314, 344, 348], [8, 18, 52, 65, 102, 110, 145, 176, 179, 185, 235, 242, 255, 279, 347], [13, 57, 77, 84, 94, 118, 152, 154, 162, 170, 187, 250, 329], [6, 21, 31, 39, 51, 105, 109, 125, 140, 150, 156, 171, 240, 306], [7, 20, 56, 85, 111, 120, 157, 167, 232, 252, 261, 340], [23, 37, 64, 127, 144, 178, 182, 191, 201, 253, 265], [35, 50, 106, 123, 141, 177, 189, 200, 219, 302, 323, 327, 331, 339], [15, 30, 33, 108, 128, 151, 165, 180, 214, 318], [16, 38, 53, 59, 98, 114, 122, 169, 251, 264, 287, 342, 345], [10, 34, 55, 126, 143, 210, 225, 246, 272, 278, 301], [17, 27, 117, 119, 129, 148, 164, 192, 197, 199, 239, 263, 307], [87, 103, 115, 132, 134, 172, 190, 196, 198, 209, 247, 330], [25, 43, 74, 83, 175, 202, 221, 259, 351], [4, 61, 133, 149, 163, 213, 227, 275, 277, 281, 295, 31

In [61]:
save_solution(solutions_4) # сохранение лучших ответов (на примере четвёртого алгоритма - Smallest degree last with remove с рандомизацией)